In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
from torch.utils.data import Dataset

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

In [ ]:
cd yolov5

In [ ]:
!pip install -r requirements.txt
!pip install torchinfo

In [ ]:
from utils.dataloaders import create_dataloader
from utils.general import (LOGGER, TQDM_BAR_FORMAT, Profile, check_dataset, check_img_size, check_requirements,
                           check_yaml, coco80_to_coco91_class, colorstr, increment_path, non_max_suppression,
                           print_args, scale_boxes, xywh2xyxy, xyxy2xywh)
import cv2
import matplotlib.pyplot as plt
from models.yolo import Model
import torch
import yaml
from utils.general import (LOGGER, TQDM_BAR_FORMAT, check_amp, check_dataset, check_file, check_git_info,
                           check_git_status, check_img_size, check_requirements, check_suffix, check_yaml, colorstr,
                           get_latest_run, increment_path, init_seeds, intersect_dicts, labels_to_class_weights,
                           labels_to_image_weights, methods, one_cycle, print_args, print_mutation, strip_optimizer,
                           yaml_save)
import torchinfo
from PIL import Image
from utils.loss import ComputeLoss
from utils.metrics import ConfusionMatrix, ap_per_class, box_iou
from utils.general import (LOGGER, TQDM_BAR_FORMAT, Profile, check_dataset, check_img_size, check_requirements,
                           check_yaml, coco80_to_coco91_class, colorstr, increment_path, non_max_suppression,
                           print_args, scale_boxes, xywh2xyxy, xyxy2xywh)
import os

In [ ]:
train_loader, dataset = create_dataloader('/kaggle/input/congapro/content/Yolo_1/images/train',
                                            640,
                                            1,
                                            32,
                                            True,
                                            pad = 0,
                                            rect = True,
                                            workers = 2,
                                            prefix = colorstr(f'train'))

In [ ]:
for (data, target, paths, _) in train_loader:
    original_data = data.squeeze().permute(1, 2, 0).numpy()
    plt.imshow(original_data)
    print(original_data.shape)
    print(paths)
    break

In [ ]:
with open('/kaggle/working/yolov5/data/hyps/hyp.scratch-low.yaml', errors = 'ignore') as f:
  hyp = yaml.safe_load(f)
device = 'cuda'
weights = '/kaggle/input/model/best.pt'

In [ ]:
ckpt = torch.load(weights, map_location='cpu')  # load checkpoint to CPU to avoid CUDA memory leak
model = Model(ckpt['model'].yaml, ch=3, nc=1, anchors=3).to(device)  # create
exclude = ['anchor'] if (hyp.get('anchors')) and not resume else []  # exclude keys
csd = ckpt['model'].float().state_dict()  # checkpoint state_dict as FP32
csd = intersect_dicts(csd, model.state_dict(), exclude=exclude)  # intersect
model.load_state_dict(csd, strict=False)  # load
LOGGER.info(f'Transferred {len(csd)}/{len(model.state_dict())} items from {weights}')  # report

In [ ]:
# FGSM attack code
def fgsm_attack(image, epsilon, data_grad):
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon*sign_data_grad
    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    # Return the perturbed image
    return perturbed_image

# restores the tensors to their original scale
def denorm(batch, mean=[0.1307], std=[0.3081]):
    """
    Convert a batch of tensors to their original scale.

    Args:
        batch (torch.Tensor): Batch of normalized tensors.
        mean (torch.Tensor or list): Mean used for normalization.
        std (torch.Tensor or list): Standard deviation used for normalization.

    Returns:
        torch.Tensor: batch of tensors without normalization applied to them.
    """
    if isinstance(mean, list):
        mean = torch.tensor(mean).to(device)
    if isinstance(std, list):
        std = torch.tensor(std).to(device)

    return batch * std.view(1, -1, 1, 1) + mean.view(1, -1, 1, 1)

In [ ]:
# from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
def test( model, device, test_loader, epsilon ):
    pbar = enumerate(test_loader)
    # Accuracy counter
    correct = 0
    adv_examples = []
    model.hyp = hyp
    compute_loss = ComputeLoss(model)

    # Loop over all examples in test set
    for i, (data, target, paths, _) in pbar:
#         print('/kaggle/working/data_fgsm/' + paths[0].split('/')[-1])
        original_data = data.squeeze().permute(1, 2, 0).numpy()
#         plt.imshow(original_data)
        # img = Image.fromarray(original_data, 'RGB')
        # img.save('/content/lucdau.png')
#         tmp = cv2.imread(paths[0])
        # print(tmp)
        # print(original_data)
#         cv2_imshow(tmp)
#         print('💩' * 50)
        # cv2_imshow(original_data)

        # Send the data and label to the device
        data = data.to(device, non_blocking=True).float() / 255  # uint8 to float32, 0-255 to 0.0-1.0
        # data = data.permute(0, 2, 3, 1).type(torch.float)
        # Set requires_grad attribute of tensor. Important for Attack
        data.requires_grad = True
        # data.retain_grad()
        target = target.to(device)
        # Forward pass the data through the model
        output = model(data)
#         print(len(output))
#         output = torch.unsqueeze(output, 0)
#         init_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability

        # If the initial prediction is wrong, don't bother attacking, just move on
#         if init_pred.item() != target.item():
#             continue
        # print(output.dtype, target.dtype)
        # Calculate the loss
        loss, loss_items = compute_loss(output, target.to(device))
        # print('💩' * 50)
        # Zero all existing gradients
        model.zero_grad()

        # Calculate gradients of model in backward pass
        loss.backward()
#         print(loss)
        # print(output.pandas().xyxy[0])
        # for si, pred in enumerate(output):
        #   print(si)
        #   labels = target[target[:, 0] == si, 1:]
        #   nl, npr = labels.shape[0], pred.shape[0]  # number of labels, predictions
        #   # path, shape = Path(paths[si]), shapes[si][0]
        #   print(nl, npr)
        #   print(labels.shape)
        #   print(pred.shape)
        #   predn = pred.clone()
        #   tbox = xywh2xyxy(labels[:, 1:5])
        #   print(labels[:, 1:].shape, predn[:, :4].shape)
        #   iou = box_iou(labels[:, 1:], predn[:, :4])
        #   print(labels[:, 1:].shape, predn[:, :4].shape)
        #   print(iou)

        # Collect ``datagrad``
        data_grad = data.grad.data

        # Restore the data to its original scale
#         data_denorm = denorm(data)
        data_denorm = torch.clone(data).to(device)
        data_denorm_numpy = data_denorm.permute(0, 2, 3, 1).squeeze().detach().cpu().numpy() * 255
        cv2.imwrite('/kaggle/working/data_fgsm/images/train/' + paths[0].split('/')[-1], cv2.cvtColor(data_denorm_numpy, cv2.COLOR_RGB2BGR))
        # cv2_imshow(cv2.cvtColor(data_denorm_numpy, cv2.COLOR_RGB2BGR))
        # print(data_denorm_numpy.shape, data_denorm_numpy)
        # img = Image.fromarray(data_denorm_numpy)
        # img.show()

        # Call FGSM Attack
        perturbed_data = fgsm_attack(data_denorm, 0.05, data_grad)
        perturbed_data_numpy = perturbed_data.permute(0, 2, 3, 1).squeeze().detach().cpu().numpy() * 255
#         plt.imshow(cv2.cvtColor(perturbed_data_numpy, cv2.COLOR_RGB2BGR))
        cv2.imwrite('/kaggle/working/data_fgsm/noised_images/train/' + paths[0].split('/')[-1], cv2.cvtColor(perturbed_data_numpy, cv2.COLOR_RGB2BGR))
        # Reapply normalization
        # perturbed_data_normalized = transforms.Normalize((0.1307,), (0.3081,))(perturbed_data)

        # Re-classify the perturbed image
        # output = model(perturbed_data)
        # # print(type(output), target)


        # # Check for success
        # final_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        # if final_pred.item() == target.item():
        #     correct += 1
        #     # Special case for saving 0 epsilon examples
        #     if epsilon == 0 and len(adv_examples) < 5:
        #         adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
        #         adv_examples.append( (init_pred.item(), final_pred.item(), adv_ex) )
        # else:
        #     # Save some adv examples for visualization later
        #     if len(adv_examples) < 5:
        #         adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
        #         adv_examples.append( (init_pred.item(), final_pred.item(), adv_ex) )

    # Calculate final accuracy for this epsilon
    # final_acc = correct/float(len(test_loader))
    # print(f"Epsilon: {epsilon}\tTest Accuracy = {correct} / {len(test_loader)} = {final_acc}")

    # # Return the accuracy and an adversarial example
    # return final_acc, adv_examples
    
    return 1, 1

In [ ]:
epsilons = [.1]
# Set random seed for reproducibility
torch.manual_seed(42)
os.makedirs('/kaggle/working/data_fgsm/',exist_ok = True)
os.makedirs('/kaggle/working/data_fgsm/images/',exist_ok = True)
os.makedirs('/kaggle/working/data_fgsm/images/train',exist_ok = True)
os.makedirs('/kaggle/working/data_fgsm/images/val',exist_ok = True)
os.makedirs('/kaggle/working/data_fgsm/noised_images/',exist_ok = True)
os.makedirs('/kaggle/working/data_fgsm/noised_images/train',exist_ok = True)
os.makedirs('/kaggle/working/data_fgsm/noised_images/val',exist_ok = True)

In [ ]:
accuracies = []
examples = []

# Run test for each epsilon
for eps in epsilons:
    model.to(device)
    acc, ex = test(model, device, train_loader, eps)
    accuracies.append(acc)
    examples.append(ex)

In [ ]:
train_loader, dataset = create_dataloader('/kaggle/input/congapro/content/Yolo_1/images/val',
                                            640,
                                            1,
                                            32,
                                            True,
                                            pad = 0,
                                            rect = True,
                                            workers = 2,
                                            prefix = colorstr(f'train'))

In [ ]:
# from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
def test( model, device, test_loader, epsilon ):
    pbar = enumerate(test_loader)
    # Accuracy counter
    correct = 0
    adv_examples = []
    model.hyp = hyp
    compute_loss = ComputeLoss(model)

    # Loop over all examples in test set
    for i, (data, target, paths, _) in pbar:
#         print('/kaggle/working/data_fgsm/' + paths[0].split('/')[-1])
        original_data = data.squeeze().permute(1, 2, 0).numpy()
#         plt.imshow(original_data)
        # img = Image.fromarray(original_data, 'RGB')
        # img.save('/content/lucdau.png')
#         tmp = cv2.imread(paths[0])
        # print(tmp)
        # print(original_data)
#         cv2_imshow(tmp)
#         print('💩' * 50)
        # cv2_imshow(original_data)

        # Send the data and label to the device
        data = data.to(device, non_blocking=True).float() / 255  # uint8 to float32, 0-255 to 0.0-1.0
        # data = data.permute(0, 2, 3, 1).type(torch.float)
        # Set requires_grad attribute of tensor. Important for Attack
        data.requires_grad = True
        # data.retain_grad()
        target = target.to(device)
        # Forward pass the data through the model
        output = model(data)
#         print(len(output))
#         output = torch.unsqueeze(output, 0)
#         init_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability

        # If the initial prediction is wrong, don't bother attacking, just move on
#         if init_pred.item() != target.item():
#             continue
        # print(output.dtype, target.dtype)
        # Calculate the loss
        loss, loss_items = compute_loss(output, target.to(device))
        # print('💩' * 50)
        # Zero all existing gradients
        model.zero_grad()

        # Calculate gradients of model in backward pass
        loss.backward()
#         print(loss)
        # print(output.pandas().xyxy[0])
        # for si, pred in enumerate(output):
        #   print(si)
        #   labels = target[target[:, 0] == si, 1:]
        #   nl, npr = labels.shape[0], pred.shape[0]  # number of labels, predictions
        #   # path, shape = Path(paths[si]), shapes[si][0]
        #   print(nl, npr)
        #   print(labels.shape)
        #   print(pred.shape)
        #   predn = pred.clone()
        #   tbox = xywh2xyxy(labels[:, 1:5])
        #   print(labels[:, 1:].shape, predn[:, :4].shape)
        #   iou = box_iou(labels[:, 1:], predn[:, :4])
        #   print(labels[:, 1:].shape, predn[:, :4].shape)
        #   print(iou)

        # Collect ``datagrad``
        data_grad = data.grad.data

        # Restore the data to its original scale
#         data_denorm = denorm(data)
        data_denorm = torch.clone(data).to(device)
        data_denorm_numpy = data_denorm.permute(0, 2, 3, 1).squeeze().detach().cpu().numpy() * 255
        cv2.imwrite('/kaggle/working/data_fgsm/images/val/' + paths[0].split('/')[-1], cv2.cvtColor(data_denorm_numpy, cv2.COLOR_RGB2BGR))
        # cv2_imshow(cv2.cvtColor(data_denorm_numpy, cv2.COLOR_RGB2BGR))
        # print(data_denorm_numpy.shape, data_denorm_numpy)
        # img = Image.fromarray(data_denorm_numpy)
        # img.show()

        # Call FGSM Attack
        perturbed_data = fgsm_attack(data_denorm, 0.05, data_grad)
        perturbed_data_numpy = perturbed_data.permute(0, 2, 3, 1).squeeze().detach().cpu().numpy() * 255
#         plt.imshow(cv2.cvtColor(perturbed_data_numpy, cv2.COLOR_RGB2BGR))
        cv2.imwrite('/kaggle/working/data_fgsm/noised_images/val/' + paths[0].split('/')[-1], cv2.cvtColor(perturbed_data_numpy, cv2.COLOR_RGB2BGR))
        # Reapply normalization
        # perturbed_data_normalized = transforms.Normalize((0.1307,), (0.3081,))(perturbed_data)

        # Re-classify the perturbed image
        # output = model(perturbed_data)
        # # print(type(output), target)


        # # Check for success
        # final_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        # if final_pred.item() == target.item():
        #     correct += 1
        #     # Special case for saving 0 epsilon examples
        #     if epsilon == 0 and len(adv_examples) < 5:
        #         adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
        #         adv_examples.append( (init_pred.item(), final_pred.item(), adv_ex) )
        # else:
        #     # Save some adv examples for visualization later
        #     if len(adv_examples) < 5:
        #         adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
        #         adv_examples.append( (init_pred.item(), final_pred.item(), adv_ex) )

    # Calculate final accuracy for this epsilon
    # final_acc = correct/float(len(test_loader))
    # print(f"Epsilon: {epsilon}\tTest Accuracy = {correct} / {len(test_loader)} = {final_acc}")

    # # Return the accuracy and an adversarial example
    # return final_acc, adv_examples
    
    return 1, 1

In [ ]:
accuracies = []
examples = []

# Run test for each epsilon
for eps in epsilons:
    model.to(device)
    acc, ex = test(model, device, train_loader, eps)
    accuracies.append(acc)
    examples.append(ex)

In [ ]:
cp -R /kaggle/input/congapro/content/Yolo_1/labels /kaggle/working/data_fgsm/labels

In [ ]:
import zipfile
import os
from IPython.display import FileLink

def zip_dir(directory = os.curdir, file_name = 'directory.zip'):
    """
    zip all the files in a directory
    
    Parameters
    _____
    directory: str
        directory needs to be zipped, defualt is current working directory
        
    file_name: str
        the name of the zipped file (including .zip), default is 'directory.zip'
        
    Returns
    _____
    Creates a hyperlink, which can be used to download the zip file)
    """
    os.chdir(directory)
    zip_ref = zipfile.ZipFile(file_name, mode='w')
    for folder, _, files in os.walk(directory):
        for file in files:
            if file_name in file:
                pass
            else:
                zip_ref.write(os.path.join(folder, file))

    return FileLink(file_name)

In [ ]:
cd ..

In [ ]:
zip_dir('/kaggle/working/data_fgsm',)